## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import gc

pd.options.mode.chained_assignment = None

In [9]:
source_path = '../data/mimiciii/source_data/'
dest_path = '../data/mimiciii/preprocess_data/'

## Split CHARTEVENTS to several dataset

In [4]:
count = 1
for reader in tqdm(pd.read_csv(source_path + 'CHARTEVENTS.csv', chunksize = 40000000)):
    reader.to_csv(dest_path + 'CHARTEVENTS' + str(count) + '.csv', index=False)
    count += 1

0it [00:00, ?it/s]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
4it [06:36, 98.71s/it] /home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
6it [09:54, 99.03s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,10,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
9it [13:37, 90.83s/it]


## Load Data

In [3]:
paths = sorted(os.listdir(source_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CHARTEVENTS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('D_CPT.csv')
paths

['ADMISSIONS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'MICROBIOLOGYEVENTS.csv',
 'NOTEEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv',
 'PROCEDUREEVENTS_MV.csv',
 'PROCEDURES_ICD.csv']

In [4]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(source_path + path))
    
for path in tqdm(sorted(os.listdir(dest_path))):
    datas.append(pd.read_csv(dest_path + path))

paths += sorted(list(os.listdir(dest_path)))

 44%|██████████████████████████████████████████████████████████████████████████▏                                                                                            | 8/18 [00:00<00:00, 29.62it/s]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,9,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 61%|█████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                | 11/18 [00:40<00:27,  3.92s/it]/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 78%|██████████████████████████████████████████████████████████████████████████████████████████████

## Clean Data to remain SEPSIS patient and sort Data

In [5]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv(dest_path + paths[i], index=False)
D_ICD_DIAGNOSES

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
10304,11403,99591,Sepsis,Sepsis
10305,11404,99592,Severe sepsis,Severe sepsis
13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [6]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv(dest_path + paths[i], index=False)
DIAGNOSES_ICD

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 651047/651047 [02:49<00:00, 3838.03it/s]


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
747,140,21,111970,11.0,99592
914,307,38,185910,3.0,99592
1084,477,61,189535,8.0,99591
1097,490,62,116009,4.0,99591
276,505,64,172056,3.0,99591
...,...,...,...,...,...
648640,650341,99836,101018,8.0,99592
649024,650470,99865,189346,4.0,99592
649529,650720,99912,189380,8.0,99592
648171,650990,99985,176670,7.0,99592


In [7]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


In [8]:
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('DIAGNOSES_ICD.csv'):
        continue
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        # datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
        # datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)

    datas[i].to_csv(dest_path + paths[i], index=False)
    
del dict1
gc.collect()
        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(dest_path + paths[i], index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [27:09<00:00, 60.35s/it]


"\nfor i in tqdm(range(len(datas))):\n    for column in datas[i].columns:\n        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:\n            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))\n"